# Hello

### Importar librerias

In [14]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import classification_report,accuracy_score
import json
import pickle

### Extraer datos

In [15]:
#path = kagglehub.dataset_download("fedesoriano/company-bankruptcy-prediction")
#data = pd.read_csv(path+"/data.csv")

In [16]:
data = pd.read_csv("../data/pca_10_componentes.csv")

In [17]:
data.head(10)


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,Bankrupt?
0,-7.338294,0.373294,-0.309014,-1.021642,0.162798,1.448300,-0.710591,1.195317,-0.535536,1.665768,1
1,-2.703713,-0.986346,-2.155617,-2.473644,0.835692,0.607047,-0.324440,1.361657,-0.409246,0.693192,1
2,-4.307059,-0.404700,-0.309801,-0.729865,0.335723,-0.198898,0.016022,0.562789,-0.501530,1.793298,1
3,-5.830736,-1.385669,-1.301105,-2.160071,0.731532,1.043664,0.558476,-0.195704,0.705732,0.024977,1
4,-2.343197,-0.201748,0.806267,-0.894912,0.203652,0.266931,0.349238,-0.201997,0.604713,-1.176532,1
5,-6.681856,-0.774171,-0.927413,-1.612984,0.432967,2.471917,0.463235,0.190965,0.567973,1.592355,1
6,-5.855967,0.082141,-0.627971,-0.033042,0.187159,1.688968,-1.135885,2.090697,-0.923283,0.750207,0
7,-0.448640,0.203617,0.487619,-1.186015,0.333833,0.580163,0.741092,-0.258178,0.830105,-1.716014,0
8,-0.530649,-1.312022,-0.004166,0.584962,0.009090,-0.724902,0.192013,-0.566441,0.488995,-0.669149,0
9,-0.366075,0.735611,1.246965,-0.469174,0.144420,1.100224,0.966121,-0.390257,1.050538,-1.697269,0


In [18]:
data.shape

(6819, 11)

### Modelo

In [19]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(["Bankrupt?"], axis=1), data["Bankrupt?"], test_size=0.2, random_state=42)
X_train

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
3759,-1.140608,0.480156,-0.759926,-2.796154,0.772126,0.727323,1.014923,-0.145113,0.590672,-0.727444
1782,-0.260267,0.572487,2.348049,2.301804,-0.597558,-0.080781,-0.040810,-0.003390,-0.015728,0.893517
5013,0.900051,-0.714695,-1.732955,-0.897534,0.216791,-0.380074,-0.364174,0.109163,-0.571839,1.182433
5412,-1.186987,3.418958,3.690440,-0.698332,-0.072059,0.024235,0.404884,0.268749,0.296243,-1.899067
3066,-1.221979,0.844584,1.145609,-0.769437,0.152215,0.142710,0.347644,-0.029464,0.163310,-0.357528
...,...,...,...,...,...,...,...,...,...,...
3772,5.085515,1.215414,1.088639,-0.240453,-0.176133,-1.321257,1.384407,-2.406528,0.726128,0.002931
5191,3.875070,0.616553,0.584459,0.515760,-0.168247,-0.524600,0.098302,-0.687327,-0.022449,-0.026363
5226,-0.117474,0.541386,1.877499,1.377089,-0.322174,0.679777,0.142341,0.085510,0.714094,-0.837932
5390,4.052295,0.495118,0.217813,0.561029,-0.272347,-0.225973,-0.274712,-0.067791,-0.161443,-0.312956


In [20]:
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [21]:
def sigmoid(x):
    return [1 / (1 + np.exp(-y)) for y in x]

y_pred = sigmoid(y_pred)

y_pred_sort = {i:y_pred[i] for i in range(len(y_pred))}
y_pred_sort = dict(sorted(y_pred_sort.items(), key=lambda x: x[1], reverse=True))

how_many_ones = y_test.value_counts()[1]

one_predictions = list(y_pred_sort.keys())[:how_many_ones]

final_preds = [1 if x in one_predictions else 0 for x in range(len(y_pred))]

In [22]:
print(classification_report(y_test, final_preds))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1313
           1       0.43      0.43      0.43        51

    accuracy                           0.96      1364
   macro avg       0.70      0.70      0.70      1364
weighted avg       0.96      0.96      0.96      1364



In [23]:
X_test

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
239,-2.508763,-3.236584,-2.634913,-0.471904,0.545137,-0.372686,0.579150,-0.970863,0.184079,0.388427
2850,0.554196,0.948523,0.697611,-1.196512,0.254458,0.489612,0.528982,-0.157908,0.467491,-1.310643
2687,3.204800,0.483832,0.528284,0.516076,-0.131221,1.569525,0.561481,-0.110738,0.914215,-1.201075
6500,-5.271985,2.967743,2.454384,1.774013,-0.483117,-0.015024,-0.965084,1.180668,-0.749741,0.226518
2684,-1.350313,0.180171,-2.083013,-2.409058,0.722842,0.991949,-0.215720,1.092622,-0.366141,0.814691
...,...,...,...,...,...,...,...,...,...,...
1357,0.514038,1.313538,1.760316,-1.034971,0.073677,0.288101,0.818204,-0.738057,0.717431,-1.290878
3946,0.981786,-1.853898,0.566422,4.260684,-0.805452,-1.585961,-0.686697,-0.611576,0.042463,-0.635425
5491,-2.099497,0.530659,-0.590708,-1.418302,0.358275,1.015674,0.036807,0.803366,-0.110907,0.524821
2112,-3.671366,1.539616,1.930818,-0.678460,0.141061,-1.495039,-0.246367,0.226574,-0.701971,-0.100105


In [24]:
accuracy = accuracy_score(y_test,final_preds)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9574780058651027


In [25]:
umbral = min([y_pred_sort[x] for x in one_predictions])

print(f"CÓMO PREDECIR CON ESTE MODELO:\n\n Paso 1.- Calcularlo con la regresión lineal.\n PASO 2.- Aplicarle sigmoide para acotarlo al intervalo [0,1].\n PASO 3.- Si el resultado es mayor a {umbral} entonces predecir 1.\n          De lo contrario, predecir 0.\n\nAsegúrense de meter esto en score.py")


umb = json.dumps({"umbral": [umbral]})
file = open("umbral.json", "w")
file.write(umb)
file.close()

file = open('model.pkl', 'wb')
pickle.dump(reg, file)
file.close()


CÓMO PREDECIR CON ESTE MODELO:

 Paso 1.- Calcularlo con la regresión lineal.
 PASO 2.- Aplicarle sigmoide para acotarlo al intervalo [0,1].
 PASO 3.- Si el resultado es mayor a 0.5320759532269486 entonces predecir 1.
          De lo contrario, predecir 0.

Asegúrense de meter esto en score.py
